In [11]:
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer # Работает только с английскими словами
from pymystem3 import Mystem
import pymorphy2
import re
import torch

In [14]:
torch.cuda.is_available()

False

In [6]:
import nltk

nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/timofey/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/timofey/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [7]:
# Загрузка данных
train_data = pd.read_csv('train.csv')

In [30]:
train_data

,id,url,title,target,text
0,0,m.kp.md,"Экс-министр экономики Молдовы - главе МИДЭИ, ц...",False,"Экс-министр экономики Молдовы - главе МИДЭИ, ц..."
1,1,www.kp.by,Эта песня стала известна многим телезрителям б...,False,Эта песня стала известна многим телезрителям б...
2,2,fanserials.tv,Банши 4 сезон 2 серия Бремя красоты смотреть о...,False,Банши 4 сезон 2 серия Бремя красоты смотреть о...
3,3,colorbox.spb.ru,Не Беси Меня Картинки,False,Не Беси Меня Картинки colorbox.spb.ru
4,4,tula-sport.ru,В Новомосковске сыграют следж-хоккеисты алекси...,False,В Новомосковске сыграют следж-хоккеисты алекси...
...,...,...,...,...,...
135304,135304,mail.ru,пора тюльпанов турецкий сериал на русском язык...,False,пора тюльпанов турецкий сериал на русском язык...
135305,135305,www.ntv.ru,Остросюжетный сериал «Шеф. Игра на повышение»....,False,Остросюжетный сериал «Шеф. Игра на повышение»....
135306,135306,topclassiccarsforsale.com,"1941 Plymouth Special Deluxe Hot Rod, Automati...",False,"1941 Plymouth Special Deluxe Hot Rod, Automati..."
135307,135307,wowcream.ru,Купить It's Skin Сыворотка питательная Power 1...,False,Купить It's Skin Сыворотка питательная Power 1...


In [178]:
lemmatizer = WordNetLemmatizer()
lemmatizer.lemmatize("бегущие")

'бегущие'

In [179]:
lemmatizer = pymorphy2.MorphAnalyzer()
lemmatizer.parse("бегущие")[0].normal_form

'бежать'

In [180]:
m = Mystem()
''.join(m.lemmatize("бегущие")).strip()

'бежать'

In [15]:
stop_words = set(stopwords.words('russian'))
# stop_words = set(stopwords.words('russian')) | set(stopwords.words('english'))

def clean_data(data):
    data.drop_duplicates(inplace=True)
    data.dropna(inplace=True)
    data['title'] = data['title'].apply(lambda x: x.lower())
    # Удаляем все, кроме слов и цифр с пробелами
    data['title'] = data['title'].apply(lambda x: re.sub(r'[^\w\s]', '', x)) 
    data['title'] = data['title'].apply(lambda x: re.sub(r'\d+', '', x)) 

    return data

### Разные лемматизации

In [12]:
def WordNetLemmatize(train_data):
    lemmatizer = WordNetLemmatizer()
    return train_data.apply(lambda x: ' '.join([lemmatizer.lemmatize(word) for word in x.split() if word not in stop_words]))

In [8]:
def MystemLemmatize(train_data):
    # Инициализация объекта для лемматизации
    mystem = Mystem()

    def preprocess_text(text):
        # Лемматизация
        lemmas = mystem.lemmatize(text)
        # Удаление стоп-слов
        lemmas = [lemma.strip() for lemma in lemmas if lemma not in stop_words]
        # Склеивание лемм в одну строку
        text = ' '.join(lemmas)
        return text

    return train_data.apply(lambda text: preprocess_text(text))

In [32]:
def pymorphy2Lemmatize(train_data):
    lemmatizer = pymorphy2.MorphAnalyzer()

    return train_data.apply(
        lambda x: ' '.join([lemmatizer.parse(word)[0].normal_form for word in x.split() if word not in stop_words])
    )

In [15]:
train_data = clean_data(train_data)

In [17]:
train_data['title'] = MystemLemmatize(train_data['title'])

### Преобразование текста в числа

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer

train_data['text'] = train_data['title'].apply(lambda x: x + ' ') + train_data['url']
# Преобразование текста в числа
vectorizer = CountVectorizer()
# vectorizer = TfidfVectorizer()

X = vectorizer.fit_transform(train_data['text'])
y = train_data['target']

# Разбиение данных на обучающую и валидационную выборки
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [188]:
feature_names = np.array(list(vectorizer.vocabulary_.keys()))
print(feature_names)

['эксминистр' 'экономика' 'молдова' ... 'blockpower' 'effector' 'wowcream']


In [143]:
id_ = 42

print(X_train[id_])

x_vector = X.getrow(id_).toarray()[0]

[feature for feature in feature_names[x_vector > 0]]

Презентация на тему "Приближенное значение. Абсолютная и относительная погрешнос


['гидромассажный',
 'olgafisakovamailru',
 'юнитекс',
 'размешать',
 'alvares',
 'grou',
 'кренк']

In [7]:
# Обучение модели
model = LogisticRegression()
model.fit(X_train, y_train)

/home/timofey/.local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

# Оценка качества модели на валидационной выборке

In [52]:
y_pred = model.predict(X_train)
score = f1_score(y_train, y_pred)
print("F1-score на тестовой выборке: {:.3f}".format(score))

y_pred_val = model.predict(X_val)
score = f1_score(y_val, y_pred_val)
print("F1-score на валидационной выборке: {:.3f}".format(score))

F1-score на тестовой выборке: 0.989
F1-score на валидационной выборке: 0.969


In [43]:
y_pred = model.predict(X_train)
score = f1_score(y_train, y_pred)
print("F1-score на тестовой выборке: {:.3f}".format(score))

y_pred_val = model.predict(X_val)
score = f1_score(y_val, y_pred_val)
print("F1-score на валидационной выборке: {:.3f}".format(score))

F1-score на тестовой выборке: 0.990
F1-score на валидационной выборке: 0.967


In [29]:
y_pred = model.predict(X_train)
score = f1_score(y_train, y_pred)
print("F1-score на тестовой выборке: {:.3f}".format(score))

y_pred_val = model.predict(X_val)
score = f1_score(y_val, y_pred_val)
print("F1-score на валидационной выборке: {:.3f}".format(score))

F1-score на тестовой выборке: 0.989
F1-score на валидационной выборке: 0.989


## Соединяем обработки текста в отдельную функцию

In [245]:
def train_model(train_data, with_report = False,
                model=LogisticRegression(),
                Lemmatizer = None, 
                stop_words = set(stopwords.words('russian')),
               ):
    # Очищаем
    train_data = clean_data(train_data)
    # Лемматизируем
    if (Lemmatizer is not None):
        train_data['title'] = Lemmatizer(train_data['title'])
    # Приводим к числам
    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(train_data['title'])
    y = train_data['target']
    
    # Разбиение данных на обучающую и валидационную выборки
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Обучение модели
    model.fit(X_train, y_train)
    
    if (with_report):
        y_pred = model.predict(X_train)
        score = f1_score(y_train, y_pred)
        print("F1-score на тестовой выборке: {:.3f}".format(score))

        y_pred_val = model.predict(X_val)
        score = f1_score(y_val, y_pred_val)
        print("F1-score на валидационной выборке: {:.3f}".format(score))
    
    return model

In [256]:
model_pymorphy2Lemmatize = train_model(train_data, with_report=True, Lemmatizer=pymorphy2Lemmatize, stop_words=stop_words_merged)

F1-score на тестовой выборке: 0.971
F1-score на валидационной выборке: 0.945


In [257]:
model_pymorphy2Lemmatize = train_model(train_data, with_report=True, Lemmatizer=pymorphy2Lemmatize)

F1-score на тестовой выборке: 0.971
F1-score на валидационной выборке: 0.945


In [258]:
model_MystemLemmatize = train_model(train_data, with_report=True, Lemmatizer=MystemLemmatize)

F1-score на тестовой выборке: 0.971
F1-score на валидационной выборке: 0.945


In [238]:
from nltk.stem import WordNetLemmatizer
model_WordNetLemmatize = train_model(train_data, with_report=True, Lemmatizer=WordNetLemmatize)

F1-score на тестовой выборке: 0.971
F1-score на валидационной выборке: 0.939


In [246]:
model_without_lemmatize = train_model(train_data, with_report=True)

F1-score на тестовой выборке: 0.971
F1-score на валидационной выборке: 0.939


## BASELINE

In [41]:
# Загрузка данных
train_data = pd.read_csv('train.csv')
train_data['title'] = pymorphy2Lemmatize(train_data['title'])
train_data = clean_data(train_data)
train_data['text'] = train_data['title'].apply(lambda x: x + ' ') + train_data['url']

In [75]:
train_data['text'] = train_data['title'].apply(lambda x: x + ' ') + 4 * train_data['url']

In [76]:
from sklearn.linear_model import SGDClassifier
from sklearn.feature_extraction.text import TfidfVectorizer

# from sklearn.feature_extraction.text import TfidfVectorizer

# Преобразование текста в числа
# vectorizer = TfidfVectorizer()
vectorizer = CountVectorizer()

X = vectorizer.fit_transform(train_data['text'])
y = train_data['target']

# Разбиение данных на обучающую и валидационную выборки
X_train = X
y_train = y
# Обучение модели
# from sklearn.ensemble import RandomForestClassifier
# model = RandomForestClassifier(class_weight="balanced")
# model.fit(X_train, y_train)


In [77]:
model2 = SGDClassifier(class_weight="balanced")
model2.fit(X_train, y_train)

SGDClassifier(class_weight='balanced')

### Submit

In [49]:
test_df = pd.read_csv("test.csv")
test_df['title'] = pymorphy2Lemmatize(test_df['title'])
test_df = clean_data(test_df)
test_df['text'] = test_df['title'].apply(lambda x: x + ' ') + test_df['url']
X_test_vectorized = vectorizer.transform(test_df["text"])

In [78]:
test_df['text'] = test_df['title'].apply(lambda x: x + ' ') + 4 * test_df['url']
X_test_vectorized = vectorizer.transform(test_df["text"])

In [58]:
X_test_vectorized.shape

(165378, 178426)

In [79]:
test_df["target"] = model2.predict(X_test_vectorized).astype(bool)

counts = test_df["target"].value_counts()
print(counts)

test_df[["id", "target"]].to_csv("ml_baseline_SGDClass_cov.csv", index=False)

!cat ml_baseline.csv | head

False    145014
True      20364
Name: target, dtype: int64
id,target
135309,False
135310,False
135311,False
135312,True
135313,False
135314,False
135315,False
135316,False
135317,False
cat: write error: Broken pipe


In [ ]:
counts = pd.read_csv("ml_baseline_outClean_merged.csv")["target"].value_counts()
print(counts)
counts = pd.read_csv("ml_baseline_CleanLemmatize_Merged_CoV.csv")["target"].value_counts()
print(counts)
counts = pd.read_csv("ml_baseline_CleanLemmatize_Merged_CoV_NoSplit.csv")["target"].value_counts()
print(counts)
print("BEST:")
counts = pd.read_csv("ml_baseline_CleanLemmatizeStem_Merged_CoV_NoSplit.csv")["target"].value_counts()
print(counts)
counts = pd.read_csv("ml_baseline.csv")["target"].value_counts()
print(counts)